In [3]:
import numpy as np

In [4]:
%load_ext fortranmagic

The fortranmagic extension is already loaded. To reload it, use:
  %reload_ext fortranmagic


In [5]:
%%fortran
subroutine geo(delay_now,&
               W_geometric,&
               ha,&
               loc,&
               dec,&
               len1)
        !declaring variables
        !Remeber intent out is variables that are sent out of the
        !subroutine..


        !
        integer(kind=4), intent(in)                 :: len1
        real(kind=8), intent(in)                    :: dec
        real(kind=8), intent(in),dimension(3)       :: loc
        real(kind=8), intent(out), dimension(len1)               :: delay_now
        real(kind=8), intent(out), dimension(len1)               :: W_geometric
        real(kind=8), intent(in), dimension(len1)                :: ha
        integer(kind=4)                    :: i
        REAL(kind=8), DIMENSION(2027520) :: X, Y, Z
        REAL(kind=8)               :: c_per_mus = 299.792458, pi = 3.14159265359
        do i = 1, len1
                X(i)            = (loc(1)*cos(dec*pi/180))/c_per_mus
                Y(i)            = (-1.0*loc(2)*cos(dec*pi/180))/c_per_mus
                Z(i)            = (loc(3)*sin(dec*pi/180))/c_per_mus
                W_geometric(i)  = (X(i)*cos(ha(i)*pi/180) +Y(i)*sin(ha(i)*pi/180) + Z(i))
                delay_now(i)    = X(i)*cos(ha(i)*pi/180) +Y(i)*sin(ha(i)*pi/180)
        enddo
        !return

end subroutine


The extension _fortran_magic_36d37276d2427940a7129fd4adf144d4 is already loaded. To reload it, use:
  %fortran_config --clean-cache


In [6]:
def get_delay(ha,
              loc,
              dec):
    """Example function for f2py
       
    """
    #Defining constants..
    c_per_mus   = 299.792458
    pi          = 3.14159265359
    deg2rad     = pi/180.0
    X           = np.zeros((len(ha)))
    Y           = np.zeros((len(ha)))
    Z           = np.zeros((len(ha)))
    W_geometric = np.zeros((len(ha)))
    delay_now   = np.zeros((len(ha)))

    for i in range(len(ha)):
        X[i]            = (loc[0] * np.cos(dec*deg2rad)) / c_per_mus
        Y[i]            = (-1.0*loc[1]*np.cos(dec*deg2rad)) / c_per_mus
        Z[i]            = (loc[2]*np.sin(dec*deg2rad))/c_per_mus
        W_geometric[i]  = (X[i]*np.cos(ha[i]*deg2rad) +Y[i]*np.sin(ha[i]*deg2rad) + Z[i])
        delay_now[i]    = X[i]*np.cos(ha[i]*deg2rad)  +Y[i]*np.sin(ha[i]*deg2rad)
    return delay_now, W_geometric


In [9]:
#defing ha and loc for benchmarking 
ha  = np.linspace(-12, 12, 10000)
loc = np.array([10, 20, 100])

# Benchmarking fortran and pure (sort of) python 

In [10]:
%timeit get_delay(ha, loc, 100)

183 ms ± 2.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit geo(ha, loc, 100)

233 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
print(f'First level improvement of ~{183e-3/233e-6}')

First level improvement of ~785.4077253218884


## Improvement of 78500 % !